In [2]:
import requests
import pandas as pd
import time 
from datetime import datetime

#Load Credentials
import json
# Load the configuration from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)


# I. Load Scores via ESPN API

In [2]:
from espn_functions import *
standings_df, matchup_df, HP_Owner, HP_Player, HP_Score, HT_Owner, HT_Score = run_espn_weekly(week = 2, year=2023)

Completed week: 1
Completed week: 2
Loaded data from ESPN for week 2 of the 2023 season
Completed processing weekly scores and standings
Saved week 2 matchup data


# II. Create GPT Summary Email

In [3]:
import openai
openai.api_key = config_data['openai']['api_key']
from tabulate import tabulate
scoring_tabulate = tabulate(matchup_df.drop(columns = 'teamId'), headers='keys', tablefmt='plain', showindex=False)
standings_tabulate = tabulate(standings_df, headers='keys', tablefmt='plain', showindex=False)

In [4]:
print(scoring_tabulate)

team_name                       matchup_id  owner            total_points  QB                 QB_PTS  RB1                    RB1_PTS  RB2                    RB2_PTS  WR1                  WR1_PTS  WR2                   WR2_PTS  TE                TE_PTS  Flex                   Flex_PTS  D/ST               D/ST_PTS  K                  K_PTS
Not Sketch                               8  Kai Shen               105.46  C.J. Stroud         21.46  Najee Harris               4.8  Alexander Mattison         3.4  A.J. Brown               4.9  Tee Higgins             24.9   David Njoku          4.8  Raheem Mostert             25.2  49ers D/ST                4  Jake Elliott          12
LI JINGPING                              8  Peter Li               101.92  Tua Tagovailoa      12.26  Bijan Robinson            19.2  James Cook                17.9  Davante Adams           17.4  Christian Kirk          16.46  Dalton Kincaid       6.8  Kendrick Bourne             4.9  Broncos D/ST              0  Riley

In [8]:
week =2
input_prompt = f"Week:{week}, \nScores Table: \n\n{scoring_tabulate}\n\nStandings Table: \n\n{standings_tabulate}\n\nSummary:"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=[
    {
      "role": "system",
      "content": "You are an AI Fantasy Football commissioner tasked with writing a weekly summary to your league mates recapping the latest week of Fantasy Football.\n\nI will provide you a weekly scores table with each matchup (rows with the same matchup_id played each other and the total_points determines the winner), the owners, their players and what they scored. There will also be a standings table with everyone's records. \nUsing this information, write an email in the style of Bill Simmons recapping the performances of teams and players. In particular, make sure to roast of the team with lowest total points). \nMake the tone funny, light-hearted and slightly sarcastic "
    },
    {
      "role": "user",
      "content": input_prompt
    }
  ],
  temperature=1,
  max_tokens=8000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response['choices'][0]['message']['content'])

Hey there, fantasy football enthusiasts!

Week 2 of our league is officially in the books, and boy, oh boy, did we have some interesting matchups. Let's dive right in and dissect the madness, shall we?

First off, let's give a standing ovation to Kai Shen and his team, Not Sketch, who managed to eke out a win with a whopping 105.46 points. I'm not sure if there was some artistic inspiration behind his team name, but his performance was anything but sketchy. Well done, Kai!

And speaking of impressive performances, our league's resident "artist" John Qian and his team, fittingly named The Real Artist, secured their second consecutive victory with a commanding score of 118.8 points. Maybe he's got a secret paint-by-numbers strategy we don't know about.

But alas, not everyone can have a victorious week. I'm sorry to say, Richie Kay, but your team, Richie's ㄱㅅㄲ s, is currently begging to be placed in the nearest dumpster after scoring a total of 108.9 points. Richie, I'm not sure what tho

# III. Send G-Mail

In [9]:
owners_dict = {
    'Peter': 'peteryushunli@gmail.com',
    'Richard': 'Richardliang2015@gmail.com',
    'Johnny': 'choi.john91@gmail.com',
    #'Henry': 'henryzhang21@gmail.com',
    'Kyle': 'komearadoc@gmail.com',
    'Scott': 'lin.scott.bhs@gmail.com',
    'JD': 'johndong2015@u.northwestern.edu',
    #'David': 'dchoi26273@gmail.com',
    'Tom': 'tzhangbhs@gmail.com',
    'Kai': 'Kaishen2@gmail.com',
    'Dan': 'jiangd93@gmail.com',
    'Andrew': 'Andydozntcare@gmail.com',
    'Justin': "jchen49@gmail.com",
    'Richie' : 'richiekayy@gmail.com',
    'John Qian': 'Ziangqian@gmail.com',
    'Francis': 'francis0jin@gmail.com'
}

In [12]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

#Format the tables
scoring_tabulate = tabulate(matchup_df, headers='keys', tablefmt='html', showindex=False)
standings_tabulate = tabulate(standings_df, headers='keys', tablefmt='html', showindex=False)

# Define message
subject = f"Weekly Fagudas FF Recap: Week {week}"
summary = response['choices'][0]['message']['content']
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p><b>Highest Scoring Team</b>: {HT_Owner}: {HT_Score} points</p>
    <p><b>Highest Scoring Player</b>: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings_tabulate}
  </body>
</html>
"""

recipients = list(owners_dict.values())
#recipients = ['lithal06@yahoo.com']
sender = "peteryushunli@gmail.com"

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection i16-20020a05620a145000b00767721aebc0sm4424494qkl.32 - gsmtp')